## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label: (None, 1)~(None, 10)
## ex. label=2 => [0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
# CNN
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())


#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy; batch_size=100,epochs=100
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=32,epochs=10)

/Users/ntust/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
/Users/ntust/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/10
50000/50000 [==============================] - 309s 6ms/step - loss: 1.3081 - accuracy: 0.5534
Epoch 2/10
50000/50000 [==============================] - 314s 6ms/step - loss: 0.8376 - accuracy: 0.7082
Epoch 3/10
50000/50000 [==============================] - 310s 6ms/step - loss: 0.6208 - accuracy: 0.7815
Epoch 4/10
50000/50000 [==============================] - 314s 6ms/step - loss: 0.4485 - accuracy: 0.8419
Epoch 5/10
50000/50000 [==============================] - 324s 6ms/step - loss: 0.3084 - accuracy: 0.8916
Epoch 6/10
50000/50000 [==============================] - 345s 7ms/step - loss: 0.2156 - accuracy: 0.9234
Epoch 7/10
50000/50000 [==============================] - 331s 7ms/step - loss: 0.1640 - accuracy: 0.9436
Epoch 8/10
50000/50000 [==============================] - 398s 8ms/step - loss: 0.1359 - accuracy: 0.9529
Epoch 9/10
50000/50000 [==============================] - 398s 8ms/step - loss: 0.1164 - accuracy: 0.9598
Epoch 10/10
50000/50000 [=====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.1248061e-04, 6.6222847e-11, 7.7517354e-03, 9.9148059e-01,
        7.4701171e-07, 1.5413461e-05, 2.8860892e-04, 5.0492970e-05,
        5.3537207e-08, 1.1671124e-10]], dtype=float32)